# Homework 4

## Question 4  
Consider a three months ATM call with strike 40 on an underlying asset with spot price 40 following a lognormal distribution with volatility 20% and paying dividends continuously at 1%. Assume the risk–free interest rate is constant at 5%.
  
(i) Compute the Black–Scholes value of the call using the routine from Table 3.1 for computing approximate values for cumulative distributions of the standard normal variable;  
(ii) Compute the Black–Scholes value of the call using Simpson’s rule with tolerance 10−12 to compute approximate values for cumulative distributions of the standard normal variable.  


### (i) Compute the Black–Scholes value of the call using the routine from Table 3.1 for computing approximate values for cumulative distributions of the standard normal variable

In [17]:
import numpy as np
import scipy.stats as spstats

# Return Call value at time 0 using Black Scholes
def C0_BS(S_0, K, r, q, T, sigma):
    d1 = (np.log(S_0/K) + (r - q + 0.5*(sigma**2)) * T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    N_d1 = spstats.norm.cdf(d1,0,1)
    N_d2 = spstats.norm.cdf(d2,0,1)
    return np.exp(-r*T) * ((S_0 * np.exp((r-q)*T) * N_d1) - (K * N_d2))

# Given values
S_0 = 40
K = 40
r = .05
q = .01
T = 3/12
sigma = .20

print(f'C_0 = {C0_BS(S_0, K, r, q, T, sigma)}')


C_0 = 1.789614929076117


### (ii) Compute the Black–Scholes value of the call using Simpson’s rule with tolerance 10−12 to compute approximate values for cumulative distributions of the standard normal variable.  

In [19]:
import numpy as np
import scipy.integrate as spi
import scipy.special as sps

# Function to approximate N using Simpsons' Rule on the definite integral
def simpsN(a,b,n):
    x = np.linspace(a,b,n+1)
    y = np.exp(-x**2/2)
    approximation = (1/2) + (1/np.sqrt(2*np.pi)) * spi.simps(y,x)
    return approximation

# Function to check tolerance and return result
def tolCheck(a,b,n):
    prev = simpsN(a,b,n)
    while True:
        n+=n
        current = simpsN(a,b,n)
        print(f'Approximation of N({b}) at interval n={n} is {current}')
        if abs(current-prev) > tolDefault:
            prev = current
        else:
            print(f'Converged at n={n} and tolerance = {abs(current-prev)} <= {tolDefault}')
            break
    
    return current
            
# Function to calculate Call value using Black Scholes with Taylor approx for N()
# Return Call value at time 0 using Black Scholes
def C0_BS(S_0, K, r, q, T, sigma, n):
    d1 = (np.log(S_0/K) + (r - q + 0.5*(sigma**2)) * T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    N_d1 = tolCheck(0,d1,n)
    N_d2 = tolCheck(0,d2,n)
    return np.exp(-r*T) * ((S_0 * np.exp((r-q)*T) * N_d1) - (K * N_d2))

# Given values
S_0 = 40
K = 40
r = .05
q = .01
T = 3/12
sigma = .20

n = 4 # starting interval value
tolDefault = 10**(-12)  # Tolerance level

print(f'C_0 = {C0_BS(S_0, K, r, q, T, sigma, n)}')


Approximation of N(0.15) at interval n=8 is 0.5596176924912438
Approximation of N(0.15) at interval n=16 is 0.5596176923778039
Approximation of N(0.15) at interval n=32 is 0.5596176923707151
Approximation of N(0.15) at interval n=64 is 0.559617692370272
Converged at n=64 and tolerance = 4.430900091279e-13 <= 1e-12
Approximation of N(0.04999999999999999) at interval n=8 is 0.5199388058388787
Approximation of N(0.04999999999999999) at interval n=16 is 0.5199388058384041
Converged at n=16 and tolerance = 4.746203430272544e-13 <= 1e-12
C_0 = 1.7896149290760468


## Homework 6

## (i) Compute and explain the difference between N(d1) and the Delta of the option;

In [3]:
import numpy as np
import scipy.stats as spstats

# Return Call value at time 0 using Black Scholes
def C_BS_param(S_0, K, r, q, T, sigma):
    d1 = (np.log(S_0/K) + (r - q + 0.5*(sigma**2)) * T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    N_d1 = spstats.norm.cdf(d1,0,1)
    call_delta = np.exp(-q*T)*N_d1
    return (N_d1, call_delta)

# Given values
S_0 = 50
K = 45
r = .03
q = .01
T = 5/12
sigma = .30

print(f'(N(d1),delta) = {C_BS_param(S_0, K, r, q, T, sigma)}')


(N(d1),delta) = (0.7529925557372731, 0.7498616140798716)


$N(d1)$ measures the rate of change of the option value wrt to change in the underlying, not taking into account dividend. $\Delta$ take into account dividend yield i.e. $\Delta = e^{-qT} N(d1)$

## (ii) Compute and explain the difference between N(d2) and the probability that the call option expires in the money.

In [5]:
import numpy as np
import scipy.stats as spstats

# Return Call value at time 0 using Black Scholes
def C_BS_param(S_0, K, r, q, T, sigma):
    d1 = (np.log(S_0/K) + (r - q + 0.5*(sigma**2)) * T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    N_d2 = spstats.norm.cdf(d2,0,1)
    p_ITM = np.exp(-q*T)*N_d2
    return (N_d2, p_ITM)

# Given values
S_0 = 50
K = 45
r = .03
q = .01
T = 5/12
sigma = .30

print(f'(N(d2),p_ITM) = {C_BS_param(S_0, K, r, q, T, sigma)}')


(N(d2),p_ITM) = (0.6880349158681413, 0.6851740679574616)


$N(d2)$ is the risk-adjusted probability that the option will be exercised, not taking into account dividend yield. In this case specifically, there is dividend yield involved.